In [1]:
import os
import sys
import time
import threading
import signal
import pickle
from datetime import datetime, timedelta
from pytube import YouTube
import requests
import select

In [2]:
class YouTubeDownloader:
    def __init__(self):
        self.yt = None
        self.video_url = None
        self.stream = None
        self.download_thread = None
        self.paused = False
        self.stop_requested = False
        self.total_size = 0
        self.downloaded_size = 0
        self.start_time = None
        self.resume_data = {}
        self.download_path = os.path.join(os.path.expanduser("~"), "Downloads")
        
        # Create downloads directory if it doesn't exist
        if not os.path.exists(self.download_path):
            os.makedirs(self.download_path)
            
        # Set up signal handling for clean exit
        signal.signal(signal.SIGINT, self.signal_handler)

    def signal_handler(self, sig, frame):
        """Handle Ctrl+C to stop the download cleanly"""
        print("\nStopping download...")
        self.stop_download()
        sys.exit(0)
        
    def fetch_video_info(self, url):
        """Get video information and available streams"""
        try:
            self.video_url = url
            self.yt = YouTube(url)
            print(f"\nVideo Title: {self.yt.title}")
            print(f"Channel: {self.yt.author}")
            print(f"Duration: {timedelta(seconds=self.yt.length)}")
            print(f"Views: {self.yt.views:,}")
            return True
        except Exception as e:
            print(f"Error fetching video info: {str(e)}")
            return False
    
    def show_available_qualities(self):
        """Display available video qualities"""
        if not self.yt:
            print("No video loaded. Please fetch video info first.")
            return []
        
        # Get progressive streams (video+audio combined)
        streams = self.yt.streams.filter(progressive=True).order_by('resolution')
        
        print("\nAvailable video qualities:")
        for i, stream in enumerate(streams, 1):
            print(f"{i}. {stream.resolution} - {self.format_size(stream.filesize)}")
        
        return streams
    
    def select_quality(self, streams, choice):
        """Select video quality based on user choice"""
        try:
            index = int(choice) - 1
            if 0 <= index < len(streams):
                self.stream = streams[index]
                self.total_size = self.stream.filesize
                print(f"Selected: {self.stream.resolution} - {self.format_size(self.total_size)}")
                return True
            else:
                print("Invalid choice.")
                return False
        except ValueError:
            print("Please enter a valid number.")
            return False
    
    def format_size(self, bytes):
        """Format file size in human-readable format"""
        for unit in ['B', 'KB', 'MB', 'GB']:
            if bytes < 1024:
                return f"{bytes:.2f} {unit}"
            bytes /= 1024
        return f"{bytes:.2f} TB"
    
    def format_time(self, seconds):
        """Format time in human-readable format"""
        if seconds < 60:
            return f"{seconds:.0f} seconds"
        elif seconds < 3600:
            minutes = seconds / 60
            return f"{minutes:.1f} minutes"
        else:
            hours = seconds / 3600
            return f"{hours:.1f} hours"
    
    def calculate_eta(self):
        """Calculate estimated time remaining"""
        if self.downloaded_size == 0:
            return "calculating..."
        
        elapsed = time.time() - self.start_time
        if elapsed == 0:
            return "calculating..."
        
        download_rate = self.downloaded_size / elapsed
        if download_rate == 0:
            return "unknown"
        
        remaining_bytes = self.total_size - self.downloaded_size
        eta_seconds = remaining_bytes / download_rate
        
        return self.format_time(eta_seconds)
    
    def download_progress_callback(self, chunk, file_handle, bytes_remaining):
        """Track download progress"""
        if self.paused:
            return
            
        self.downloaded_size = self.total_size - bytes_remaining
        progress = (self.downloaded_size / self.total_size) * 100
        speed = self.downloaded_size / (time.time() - self.start_time) if time.time() > self.start_time else 0
        
        # Clear previous line and update progress
        sys.stdout.write("\r")
        sys.stdout.write(f"Progress: {progress:.1f}% | " +
                        f"Size: {self.format_size(self.downloaded_size)}/{self.format_size(self.total_size)} | " +
                        f"Speed: {self.format_size(speed)}/s | " +
                        f"ETA: {self.calculate_eta()}")
        sys.stdout.flush()
    
    def save_resume_data(self, filename, downloaded_bytes):
        """Save download state for resuming later"""
        data = {
            'url': self.video_url,
            'filename': filename,
            'downloaded_bytes': downloaded_bytes,
            'total_size': self.total_size,
            'itag': self.stream.itag
        }
        
        with open(f"{filename}.resume", "wb") as f:
            pickle.dump(data, f)
    
    def custom_download(self):
        """Download with progress tracking and pause/resume support"""
        if not self.stream:
            print("No stream selected. Please select quality first.")
            return
            
        # Generate filename from video title
        filename = f"{self.yt.title.replace(' ', '_')}.{self.stream.subtype}"
        filename = "".join(c for c in filename if c.isalnum() or c in "._- ")
        filepath = os.path.join(self.download_path, filename)
        
        # Check for resume data
        resume_file = f"{filepath}.resume"
        initial_bytes = 0
        
        if os.path.exists(resume_file):
            try:
                with open(resume_file, "rb") as f:
                    self.resume_data = pickle.load(f)
                    
                if self.resume_data['url'] == self.video_url and self.resume_data['itag'] == self.stream.itag:
                    initial_bytes = self.resume_data['downloaded_bytes']
                    self.downloaded_size = initial_bytes
                    print(f"\nResuming download from {self.format_size(initial_bytes)}...")
            except Exception as e:
                print(f"Error loading resume data: {str(e)}")
        
        self.start_time = time.time()
        self.paused = False
        self.stop_requested = False
        
        # Start download in a separate thread
        self.download_thread = threading.Thread(
            target=self._download_thread, 
            args=(filepath, initial_bytes)
        )
        self.download_thread.start()
    
    def _download_thread(self, filepath, initial_bytes):
        """Thread function that handles the actual download"""
        try:
            file_mode = 'ab' if initial_bytes > 0 else 'wb'
            
            # Get video URL
            stream_url = self.stream.url
            
            # Setup HTTP request with range header for resume
            headers = {'Range': f'bytes={initial_bytes}-'} if initial_bytes > 0 else {}
            response = requests.get(stream_url, headers=headers, stream=True)
            
            # Calculate total size
            if initial_bytes == 0:
                self.total_size = int(response.headers.get('Content-Length', 0))
            else:
                self.total_size = initial_bytes + int(response.headers.get('Content-Length', 0))
            
            chunk_size = 1024 * 1024  # 1MB chunks
            
            with open(filepath, file_mode) as f:
                for chunk in response.iter_content(chunk_size=chunk_size):
                    if self.stop_requested:
                        # Save resume data and exit
                        self.save_resume_data(filepath, self.downloaded_size)
                        return
                        
                    while self.paused:
                        if self.stop_requested:
                            self.save_resume_data(filepath, self.downloaded_size)
                            return
                        time.sleep(0.5)  # Sleep while paused
                    
                    if chunk:
                        f.write(chunk)
                        self.downloaded_size += len(chunk)
                        # Update progress
                        progress = (self.downloaded_size / self.total_size) * 100
                        speed = self.downloaded_size / (time.time() - self.start_time) if time.time() > self.start_time else 0
                        
                        sys.stdout.write("\r")
                        sys.stdout.write(f"Progress: {progress:.1f}% | " +
                                        f"Size: {self.format_size(self.downloaded_size)}/{self.format_size(self.total_size)} | " +
                                        f"Speed: {self.format_size(speed)}/s | " +
                                        f"ETA: {self.calculate_eta()}")
                        sys.stdout.flush()
            
            # Clean up resume file since download is complete
            if os.path.exists(f"{filepath}.resume"):
                os.remove(f"{filepath}.resume")
                
            sys.stdout.write("\r")
            sys.stdout.write(f"Download completed: {filepath}\n")
            
        except Exception as e:
            print(f"\nDownload error: {str(e)}")
            self.save_resume_data(filepath, self.downloaded_size)
    
    def pause_download(self):
        """Pause the download"""
        if self.download_thread and self.download_thread.is_alive():
            self.paused = True
            print("\nDownload paused. Press 'r' to resume.")
    
    def resume_download(self):
        """Resume the download"""
        if self.download_thread and self.download_thread.is_alive():
            self.paused = False
            print("\nDownload resumed.")
    
    def stop_download(self):
        """Stop the download and save resume data"""
        if self.download_thread and self.download_thread.is_alive():
            self.stop_requested = True
            self.paused = False
            self.download_thread.join()
            print("\nDownload stopped.")



In [4]:
def main():
    downloader = YouTubeDownloader()
    
    # Get YouTube URL
    url = input("Enter YouTube URL: ")
    
    # Fetch video info
    if not downloader.fetch_video_info(url):
        return
        
    # Show available qualities and get user selection
    streams = downloader.show_available_qualities()
    if not streams:
        return
        
    choice = input("\nSelect quality (enter number): ")
    if not downloader.select_quality(streams, choice):
        return
    
    # Start download
    print("\nStarting download...")
    print("Controls: [p] pause, [r] resume, [s] stop, [q] quit")
    downloader.custom_download()
    
    # Control loop
    try:
        while downloader.download_thread.is_alive():
            if sys.stdin in select.select([sys.stdin], [], [], 0)[0]:
                key = sys.stdin.read(1)
                
                if key == 'p':
                    downloader.pause_download()
                elif key == 'r':
                    downloader.resume_download()
                elif key == 's' or key == 'q':
                    downloader.stop_download()
                    if key == 'q':
                        break
            
            time.sleep(0.1)
        
    except Exception:
        # For environments where select doesn't work with stdin
        downloader.download_thread.join()

if __name__ == "__main__":
    # Fix for Windows console
    

    orig_select = select.select  # keep reference to original select

    if os.name == 'nt':
        import msvcrt
        # Override select.select for Windows without causing recursion
        def win_select(rlist, wlist, xlist, timeout=None):
            if sys.stdin in rlist:
                if msvcrt.kbhit():
                    return [sys.stdin], [], []
                return [], [], []
            return orig_select(rlist, wlist, xlist, timeout)

        select.select = win_select

    main()
    

Error fetching video info: Exception while accessing title of https://youtube.com/watch?v=4rrlG4vn_YA. Please file a bug report at https://github.com/pytube/pytube
